In [ ]:
!pip install bs4
!pip install requests
!pip install pandas
!pip install numpy

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [ ]:
URL = "https://www.amazon.in/s?k=football&crid=2L2Y02TRLF1TX&sprefix=footbal%2Caps%2C286&ref=nb_sb_noss_2"
HEADERS = ({'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 'Accept-Language' : 'en-US, en, q=0.5'})
webpage = requests.get(URL, headers=HEADERS)

In [ ]:
def get_title(soup):

    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        title_value = title.text
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("span", attrs={'class':'a-pricee'}).string.strip()

        except:
            price = ""

    return price

def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

def get_review_count(soup):
    try:
       review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available


In [ ]:
if __name__ == '__main__':
  soup = BeautifulSoup(webpage.content, "html.parser")
  
links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

links_list = []

for link in links:
  links_list.append(link.get('href'))

d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

for link in links_list:
  product = requests.get("https://www.amazon.in" + link, headers = HEADERS)
  new_soup = soup = BeautifulSoup(product.content, "html.parser")
  d['title'].append(get_title(new_soup))
  d['price'].append(get_price(new_soup))
  d['rating'].append(get_rating(new_soup))
  d['reviews'].append(get_review_count(new_soup))
  d['availability'].append(get_availability(new_soup))
  
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [40]:
amazon_df

,title,price,rating,reviews,availability
4,Nivia Storm Football - Size 5,,3.9 out of 5 stars,"38,056 ratings",In stock
5,Nivia Home Play Mini Football/PVC Material/Foo...,,3.6 out of 5 stars,"2,418 ratings",In stock
6,Nivia Country Colour Rubber Moulded Football,,3.8 out of 5 stars,"4,175 ratings",Not Available
7,Nivia Storm Football - Size 5,,3.9 out of 5 stars,"38,056 ratings",In stock
8,"Cosco 14051 Thunder Rubber Football, Size 5, (...",,4.0 out of 5 stars,495 ratings,Only 2 left in stock
...,...,...,...,...,...
66,Vicky Oro Football Hand Sewn Synthetic 2-Ply D...,,5.0 out of 5 stars,7 ratings,In stock
67,Adidas AL RIHLA League Football - FIFA 2022 - ...,,4.5 out of 5 stars,681 ratings,In stock
68,"Nivia Super Synthetic PU Football (Size: 5, Co...",,4.0 out of 5 stars,69 ratings,In stock
69,Cosco Hurricane Football - Size 5,,4.1 out of 5 stars,41 ratings,In stock
